In [1]:
import numpy as np
import pandas as pd
import cv2
import ast
from skimage.feature import hog
from sklearn.decomposition import PCA
import ipyvolume as ipv
from sklearn import metrics

# Training SVM with HOG features for Two Wheeler Detection.......

# Creating dataset.......

In [2]:
dataset = pd.read_csv('single lane (1).csv')
data = dataset[['filename','region_shape_attributes','region_attributes']]

In [3]:
xCoordinate = []
height = []
yCoordinate = []
width = []
MotorCycle = []
Driver_Helmet = []
for z in data.region_shape_attributes:
    res = ast.literal_eval(z)
    xCoordinate.append(res['x']) 
    height.append(res['height'])    
    yCoordinate.append(res['y'])
    width.append(res['width'])

for i in data.region_attributes:
    res1 = ast.literal_eval(i)
    MotorCycle.append(res1['MotorCycle'])
    Driver_Helmet.append(res1['Driver Helmet'])

In [4]:
data.head()

,filename,region_shape_attributes,region_attributes
0,scene00001.png,"{""name"":""rect"",""x"":347,""y"":248,""width"":82,""hei...","{""MotorCycle"":""Yes"",""Driver Helmet"":""No""}"
1,scene00001.png,"{""name"":""rect"",""x"":447,""y"":163,""width"":85,""hei...","{""MotorCycle"":""Yes"",""Driver Helmet"":""Yes""}"
2,scene00001.png,"{""name"":""rect"",""x"":146,""y"":612,""width"":141,""he...","{""MotorCycle"":""Yes"",""Driver Helmet"":""No""}"
3,scene00031.png,"{""name"":""rect"",""x"":99,""y"":591,""width"":119,""hei...","{""MotorCycle"":""Yes"",""Driver Helmet"":""No""}"
4,scene00031.png,"{""name"":""rect"",""x"":462,""y"":379,""width"":80,""hei...","{""MotorCycle"":""Yes"",""Driver Helmet"":""Yes""}"


In [5]:
df1 = pd.DataFrame({"filename":data['filename'],"x_coor":xCoordinate,"y_coor":yCoordinate,
                    "height":height,"width":width,"MotorCycle":MotorCycle,"Driver Helmet":Driver_Helmet})

In [9]:
df1.head()

,filename,x_coor,y_coor,height,width,MotorCycle,Driver Helmet
0,scene00001.png,347,248,121,82,Yes,No
1,scene00001.png,447,163,100,85,Yes,Yes
2,scene00001.png,146,612,104,141,Yes,No
3,scene00031.png,99,591,122,119,Yes,No
4,scene00031.png,462,379,162,80,Yes,Yes


In [7]:
#df1.to_csv('single_lane_dataset_1.csv')

# Extracting Hog features.......

In [6]:
df1 = pd.read_csv('single_lane_dataset_1.csv')

In [7]:
df1.head()

,Unnamed: 0,filename,x_coor,y_coor,height,width,MotorCycle,Driver Helmet
0,0,scene00001.png,347,248,121,82,Yes,No
1,1,scene00001.png,447,163,100,85,Yes,Yes
2,2,scene00001.png,146,612,104,141,Yes,No
3,3,scene00031.png,99,591,122,119,Yes,No
4,4,scene00031.png,462,379,162,80,Yes,Yes


In [8]:
df1 = np.array(df1)
print(df1)

[[0 'scene00001.png' 347 ... 82 'Yes' 'No']
 [1 'scene00001.png' 447 ... 85 'Yes' 'Yes']
 [2 'scene00001.png' 146 ... 141 'Yes' 'No']
 ...
 [679 'scene09811.png' 238 ... 104 'Yes' 'Yes']
 [680 'scene09841.png' 395 ... 67 'Yes' 'Yes']
 [681 'scene09871.png' 461 ... 42 'Yes' 'Yes']]


In [9]:
hog_features = []
hog_images = []
for i in df1:
    path = 'Experi_imag/' + i[1]
    img = cv2.imread(path,0)
    print(path)
    filter_img = cv2.GaussianBlur(img,(5,5),100)
    #crop = filter_img[y_coor[i]:(y_coor[i]+height[i]),x_coor[i]:(x_coor[i]+width[i])]
    crop = filter_img[(i[3]+int(0.25*i[4])):(i[3]+i[4]),i[2]:(i[2]+i[5])]
    print(i)
    resized = cv2.resize(crop,(64,128),interpolation = cv2.INTER_AREA)
    hf,hi = hog(resized, orientations=9, pixels_per_cell=(8,8),cells_per_block=(2, 2),visualize = True)
    hog_images.append(hi)
    hog_features.append(hf)
    cv2.destroyAllWindows()

Experi_imag/scene00001.png
[0 'scene00001.png' 347 248 121 82 'Yes' 'No']
Experi_imag/scene00001.png
[1 'scene00001.png' 447 163 100 85 'Yes' 'Yes']
Experi_imag/scene00001.png
[2 'scene00001.png' 146 612 104 141 'Yes' 'No']
Experi_imag/scene00031.png
[3 'scene00031.png' 99 591 122 119 'Yes' 'No']
Experi_imag/scene00031.png
[4 'scene00031.png' 462 379 162 80 'Yes' 'Yes']
Experi_imag/scene00031.png
[5 'scene00031.png' 349 321 155 98 'Yes' 'No']
Experi_imag/scene00031.png
[6 'scene00031.png' 430 137 104 59 'Yes' 'No']
Experi_imag/scene00031.png
[7 'scene00031.png' 499 78 92 45 'Yes' 'Yes']
Experi_imag/scene00061.png
[8 'scene00061.png' 324 297 139 83 'Yes' 'No']
Experi_imag/scene00061.png
[9 'scene00061.png' 544 159 107 55 'Yes' 'Yes']
Experi_imag/scene00061.png
[10 'scene00061.png' 447 159 103 72 'Yes' 'No']
Experi_imag/scene00061.png
[11 'scene00061.png' 478 58 64 59 'Yes' 'No']
Experi_imag/scene00091.png
[12 'scene00091.png' 184 497 172 104 'Yes' 'No']
Experi_imag/scene00091.png
[13 's

Experi_imag/scene00781.png
[110 'scene00781.png' 549 163 117 67 'Yes' 'No']
Experi_imag/scene00811.png
[111 'scene00811.png' 148 376 119 72 'No' 'No']
Experi_imag/scene00811.png
[112 'scene00811.png' 56 434 139 62 'No' 'No']
Experi_imag/scene00841.png
[113 'scene00841.png' 439 311 159 149 'No' 'No']
Experi_imag/scene00841.png
[114 'scene00841.png' 273 585 123 108 'Yes' 'No']
Experi_imag/scene00871.png
[115 'scene00871.png' 466 186 105 64 'Yes' 'No']
Experi_imag/scene00871.png
[116 'scene00871.png' 535 107 104 110 'No' 'No']
Experi_imag/scene00901.png
[117 'scene00901.png' 487 238 180 157 'No' 'No']
Experi_imag/scene00931.png
[118 'scene00931.png' 307 425 154 95 'Yes' 'No']
Experi_imag/scene00931.png
[119 'scene00931.png' 557 60 109 98 'No' 'No']
Experi_imag/scene00961.png
[120 'scene00961.png' 451 139 90 62 'Yes' 'No']
Experi_imag/scene00961.png
[121 'scene00961.png' 184 298 118 51 'No' 'No']
Experi_imag/scene00961.png
[122 'scene00961.png' 258 257 113 48 'No' 'No']
Experi_imag/scene01

Experi_imag/scene02731.png
[220 'scene02731.png' 570 137 139 108 'No' 'No']
Experi_imag/scene02731.png
[221 'scene02731.png' 508 98 91 51 'Yes' 'No']
Experi_imag/scene02761.png
[222 'scene02761.png' 363 293 127 82 'Yes' 'Yes']
Experi_imag/scene02761.png
[223 'scene02761.png' 444 190 99 60 'Yes' 'No']
Experi_imag/scene02761.png
[224 'scene02761.png' 614 30 94 81 'No' 'No']
Experi_imag/scene02791.png
[225 'scene02791.png' 533 230 150 146 'No' 'No']
Experi_imag/scene02791.png
[226 'scene02791.png' 472 137 83 51 'Yes' 'Yes']
Experi_imag/scene02821.png
[227 'scene02821.png' 603 72 98 95 'No' 'No']
Experi_imag/scene02821.png
[228 'scene02821.png' 532 58 71 31 'Yes' 'Yes']
Experi_imag/scene02851.png
[229 'scene02851.png' 345 330 140 80 'Yes' 'No']
Experi_imag/scene02851.png
[230 'scene02851.png' 526 216 145 132 'No' 'No']
Experi_imag/scene02881.png
[231 'scene02881.png' 483 347 171 151 'No' 'No']
Experi_imag/scene02881.png
[232 'scene02881.png' 583 77 95 87 'No' 'No']
Experi_imag/scene02881.p

Experi_imag/scene05311.png
[330 'scene05311.png' 316 420 141 83 'Yes' 'Yes']
Experi_imag/scene05311.png
[331 'scene05311.png' 259 538 164 85 'Yes' 'Yes']
Experi_imag/scene05311.png
[332 'scene05311.png' 194 443 149 73 'Yes' 'No']
Experi_imag/scene05311.png
[333 'scene05311.png' 440 185 96 50 'Yes' 'No']
Experi_imag/scene05311.png
[334 'scene05311.png' 488 155 89 49 'Yes' 'Yes']
Experi_imag/scene05311.png
[335 'scene05311.png' 543 112 92 87 'No' 'No']
Experi_imag/scene05311.png
[336 'scene05311.png' 499 23 58 41 'Yes' 'No']
Experi_imag/scene05341.png
[337 'scene05341.png' 291 424 145 87 'Yes' 'Yes']
Experi_imag/scene05341.png
[338 'scene05341.png' 405 460 240 209 'No' 'No']
Experi_imag/scene05341.png
[339 'scene05341.png' 469 244 109 55 'Yes' 'No']
Experi_imag/scene05341.png
[340 'scene05341.png' 354 230 92 51 'Yes' 'No']
Experi_imag/scene05341.png
[341 'scene05341.png' 420 190 92 48 'Yes' 'Yes']
Experi_imag/scene05341.png
[342 'scene05341.png' 566 26 72 69 'No' 'No']
Experi_imag/scene0

Experi_imag/scene06031.png
[438 'scene06031.png' 426 397 214 166 'No' 'No']
Experi_imag/scene06031.png
[439 'scene06031.png' 503 226 134 128 'No' 'No']
Experi_imag/scene06031.png
[440 'scene06031.png' 345 261 125 67 'Yes' 'No']
Experi_imag/scene06031.png
[441 'scene06031.png' 420 93 81 41 'Yes' 'No']
Experi_imag/scene06031.png
[442 'scene06031.png' 562 63 104 92 'No' 'No']
Experi_imag/scene06061.png
[443 'scene06061.png' 519 186 144 96 'No' 'No']
Experi_imag/scene06061.png
[444 'scene06061.png' 135 543 173 110 'Yes' 'No']
Experi_imag/scene06091.png
[445 'scene06091.png' 485 273 139 76 'Yes' 'No']
Experi_imag/scene06091.png
[446 'scene06091.png' 342 255 122 81 'Yes' 'No']
Experi_imag/scene06121.png
[447 'scene06121.png' 397 299 132 83 'Yes' 'No']
Experi_imag/scene06121.png
[448 'scene06121.png' 538 107 94 58 'Yes' 'No']
Experi_imag/scene06121.png
[449 'scene06121.png' 428 121 91 58 'Yes' 'No']
Experi_imag/scene06151.png
[450 'scene06151.png' 479 121 91 64 'Yes' 'No']
Experi_imag/scene06

Experi_imag/scene07861.png
[548 'scene07861.png' 336 349 123 74 'Yes' 'No']
Experi_imag/scene07861.png
[549 'scene07861.png' 508 130 91 41 'Yes' 'Yes']
Experi_imag/scene07861.png
[550 'scene07861.png' 444 27 87 45 'No' 'No']
Experi_imag/scene07891.png
[551 'scene07891.png' 448 279 131 63 'Yes' 'Yes']
Experi_imag/scene07891.png
[552 'scene07891.png' 325 295 134 71 'Yes' 'Yes']
Experi_imag/scene07891.png
[553 'scene07891.png' 365 519 176 91 'Yes' 'No']
Experi_imag/scene07891.png
[554 'scene07891.png' 273 415 157 86 'Yes' 'No']
Experi_imag/scene07891.png
[555 'scene07891.png' 476 365 181 162 'No' 'No']
Experi_imag/scene07891.png
[556 'scene07891.png' 437 189 100 56 'Yes' 'No']
Experi_imag/scene07921.png
[557 'scene07921.png' 488 374 195 172 'No' 'No']
Experi_imag/scene07921.png
[558 'scene07921.png' 263 507 184 109 'Yes' 'Yes']
Experi_imag/scene07921.png
[559 'scene07921.png' 512 234 118 59 'Yes' 'Yes']
Experi_imag/scene07921.png
[560 'scene07921.png' 569 160 123 104 'No' 'No']
Experi_ima

Experi_imag/scene09091.png
[657 'scene09091.png' 569 69 121 96 'No' 'No']
Experi_imag/scene09121.png
[658 'scene09121.png' 250 434 171 104 'Yes' 'Yes']
Experi_imag/scene09121.png
[659 'scene09121.png' 434 175 104 68 'Yes' 'No']
Experi_imag/scene09121.png
[660 'scene09121.png' 556 159 112 63 'Yes' 'Yes']
Experi_imag/scene09121.png
[661 'scene09121.png' 591 73 87 58 'Yes' 'Yes']
Experi_imag/scene09121.png
[662 'scene09121.png' 519 85 90 58 'Yes' 'Yes']
Experi_imag/scene09151.png
[663 'scene09151.png' 404 230 118 76 'Yes' 'Yes']
Experi_imag/scene09151.png
[664 'scene09151.png' 480 81 99 59 'Yes' 'No']
Experi_imag/scene09181.png
[665 'scene09181.png' 469 119 104 64 'Yes' 'Yes']
Experi_imag/scene09181.png
[666 'scene09181.png' 523 30 77 48 'Yes' 'No']
Experi_imag/scene09211.png
[667 'scene09211.png' 521 58 72 45 'Yes' 'Yes']
Experi_imag/scene09271.png
[668 'scene09271.png' 271 389 177 107 'Yes' 'No']
Experi_imag/scene09301.png
[669 'scene09301.png' 420 185 125 74 'Yes' 'No']
Experi_imag/sce

In [10]:
hog_features = np.array(hog_features)

In [11]:
df2 = pd.DataFrame(hog_features)

In [12]:
MotorVehicle = []
for m in MotorCycle:
    if m == 'Yes':
        MotorVehicle.append(1)
    else:
        MotorVehicle.append(0)
        
MotorVehicle = np.array(MotorVehicle)

In [13]:
df2['MotorVehicle'] = MotorVehicle

In [14]:
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,3771,3772,3773,3774,3775,3776,3777,3778,3779,MotorVehicle
0,0.369926,0.216900,0.095444,0.011491,0.038315,0.011491,0.012041,0.004760,0.026007,0.369926,...,0.155598,0.028994,0.110025,0.028994,0.350096,0.127986,0.055012,0.000000,0.000000,1
1,0.124693,0.060799,0.098858,0.000000,0.045231,0.000000,0.000000,0.009195,0.000000,0.474475,...,0.300639,0.265382,0.130717,0.195196,0.206488,0.154912,0.157947,0.300639,0.031436,1
2,0.353609,0.101121,0.063955,0.025280,0.101751,0.075841,0.047966,0.091324,0.139843,0.353609,...,0.137770,0.016404,0.000000,0.000000,0.017260,0.014965,0.012227,0.076137,0.171552,1
3,0.331317,0.279131,0.005429,0.000000,0.024953,0.000000,0.002715,0.002146,0.000000,0.379280,...,0.241216,0.008661,0.027660,0.006124,0.007747,0.010455,0.008217,0.019090,0.041913,1
4,0.188796,0.000000,0.008190,0.003237,0.005791,0.000000,0.002048,0.006475,0.000000,0.400964,...,0.036492,0.023314,0.066352,0.169034,0.245017,0.000000,0.022117,0.000000,0.000000,1


In [15]:
df2.to_csv('single_lane_Vehicle_hog.csv')

In [16]:
df2 = np.array(df2)

In [17]:
features = df2[:,:-1]
print(features)

[[0.36992578 0.21690046 0.09544433 ... 0.05501227 0.         0.        ]
 [0.12469271 0.06079868 0.09885769 ... 0.15794684 0.30063911 0.03143593]
 [0.35360889 0.101121   0.06395453 ... 0.01222703 0.07613664 0.17155156]
 ...
 [0.19742463 0.04579693 0.02198129 ... 0.07802732 0.00941842 0.0173667 ]
 [0.38031541 0.06100467 0.00263686 ... 0.10891777 0.         0.        ]
 [0.24479975 0.2843973  0.02341827 ... 0.         0.         0.        ]]


# plotting data points........

In [19]:
pca = PCA(n_components = 3)
principalComponent = pca.fit_transform(features)
principalDf = pd.DataFrame(data = principalComponent
             , columns = ['Motorcycle principal component 1', 'Motorcycle principal component 2', 'Motorcycle principal component 3'])

In [20]:
principalDf['Motorcycle'] = MotorVehicle
m = principalDf[principalDf['Motorcycle'] == 1]

In [21]:
fig = ipv.figure()
ipv.scatter(principalDf['Motorcycle principal component 1'],principalDf['Motorcycle principal component 2'],principalDf['Motorcycle principal component 3'])
ipv.scatter(m['Motorcycle principal component 1'],m['Motorcycle principal component 2'],m['Motorcycle principal component 3'],color = 'blue',size = 4)
ipv.show()

# SVM training.....

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(hog_features,MotorVehicle,test_size = 0.3, random_state = 109)

In [18]:
from sklearn import svm
#creating svm classifier with kernal type as linear
clf = svm.SVC(kernel = 'linear')
# training the model using training set
clf.fit(x_train,y_train)
# predicting the output for test_set
y_pred = clf.predict(x_test)

In [19]:
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print('w = ',clf.coef_)
print('b = ',clf.intercept_)
print('Indices of support vectors = ', clf.support_)
print('Support vectors = ', clf.support_vectors_)
print('Number of support vectors for each class = ', clf.n_support_)
print('Coefficients of the support vector in the decision function = ', np.abs(clf.dual_coef_))

Accuracy: 0.9902439024390244
w =  [[-0.04723709  0.0184231  -0.00474583 ...  0.01739275  0.0013154
   0.0121936 ]]
b =  [-0.66556149]
Indices of support vectors =  [  4  19  48  59  68  85  92 125 128 161 174 175 177 217 222 226 233 239
 255 275 277 285 286 288 293 295 297 300 309 318 323 342 354 359 361 372
 374 381 384 414 418 435 457   0   6   9  10  12  17  18  28  42  47  49
  50  51  54  55  64  71  73  74  75  77  93 101 103 112 114 118 119 121
 123 136 141 145 146 153 159 167 184 185 188 189 195 198 199 202 203 204
 208 210 211 221 223 229 231 238 243 247 260 268 269 270 273 292 304 314
 322 324 331 333 334 346 349 362 366 369 393 397 398 400 405 425 442 446
 448 458]
Support vectors =  [[0.28546944 0.15645206 0.13726204 ... 0.06312302 0.0920703  0.02822947]
 [0.47862566 0.         0.00487136 ... 0.00789291 0.         0.        ]
 [0.36519647 0.07973884 0.01752131 ... 0.         0.         0.        ]
 ...
 [0.4653749  0.06269497 0.00497978 ... 0.00594353 0.00939754 0.10075886]

# Training SVM with HSV image histogram for helmet detection..... 

In [21]:
helmet_data = pd.read_csv('single_lane_helmet_hog_features.csv')

In [22]:
helmet_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,3771,3772,3773,3774,3775,3776,3777,3778,3779,Helmet
0,0,0.471778,0.027244,0.000000,0.0,0.012184,0.0,0.034461,0.054488,0.000000,...,0.489373,0.053429,0.011264,0.0,0.000000,0.0,0.022527,0.000000,0.000000,No
1,1,0.478791,0.035578,0.045003,0.0,0.000000,0.0,0.045003,0.035578,0.000000,...,0.480235,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,Yes
2,2,0.494574,0.019872,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.076062,...,0.465382,0.031586,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.189318,No
3,3,0.512448,0.000000,0.012371,0.0,0.021869,0.0,0.000000,0.029341,0.018034,...,0.438008,0.033322,0.000000,0.0,0.005405,0.0,0.000000,0.012085,0.195399,No
4,4,0.352850,0.000000,0.110349,0.0,0.156058,0.0,0.110349,0.000000,0.000000,...,0.411617,0.000000,0.185272,0.0,0.037431,0.0,0.052935,0.000000,0.000000,Yes


In [23]:
helmet_data = np.array(helmet_data)

In [25]:
helmet_features = helmet_data[:,1:-1]
helmet_variables = helmet_data[:,3781]

In [29]:
var = []
for i in helmet_variables:
    if i == 'Yes':
        var.append(1)
    else:
        var.append(0)
var = np.array(var)

In [31]:
from sklearn.model_selection import train_test_split
x_train_helmet, x_test_helmet, y_train_helmet, y_test_helmet = train_test_split(helmet_features,var,test_size = 0.3, random_state = 109)

In [32]:
from sklearn import svm
#creating svm classifier with kernal type as linear
clf_helmet = svm.SVC(kernel = 'poly', degree = 3)
# training the model using training set
clf_helmet.fit(x_train_helmet,y_train_helmet)
# predicting the output for test_set
y_pred_helmet = clf_helmet.predict(x_test_helmet)

In [33]:
print("Accuracy:",metrics.accuracy_score(y_test_helmet, y_pred_helmet))

Accuracy: 0.6821192052980133


# Execution.......

In [35]:
def MotorVehicle_detection(vehicle,classifier):
    resized = cv2.resize(vehicle,(64,128),interpolation = cv2.INTER_AREA)
    f,i = hog(resized, orientations=9, pixels_per_cell=(8,8),cells_per_block=(2, 2),visualize = True)
    result = classifier.predict(f.reshape(1,-1))
    return result

In [34]:
def Helmet_detection(helmet_portion,classifier):
#   main code..... 
    filter_img = cv2.GaussianBlur(helmet_portion,(5,5),100)
    resized = cv2.resize(filter_img,(64,128),interpolation = cv2.INTER_AREA)
    hf,_ = hog(resized, orientations=9, pixels_per_cell=(8,8),cells_per_block=(2, 2),visualize = True)
    result = classifier.predict([hf])
    return result

In [38]:
cap = cv2.VideoCapture('T2.mp4')
fgbg = cv2.createBackgroundSubtractorMOG2()
while(1):
    _,frame = cap.read()
    image = frame[0:frame.shape[0],0:int(frame.shape[1]-(frame.shape[1]*0.45))]
    cv2.imshow('img',image)
    filter_img = cv2.GaussianBlur(image,(5,5),100)
    fgmask = fgbg.apply(filter_img)
    contours, hierarchy = cv2.findContours(fgmask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    for c in contours:
        area = cv2.contourArea(c)
        if 5000<area<10000:
            x,y,w,h = cv2.boundingRect(c)
#           detecting helmet.......
            vehicle = image[(y+int(h*0.25)):(y+h),x:(x+w)]
            helmet_portion = vehicle[0:(0+int(h*0.25)),0:(0+w)]
#            motorVehicle detection.....
            result_bike = MotorVehicle_detection(vehicle,clf)

            if result_bike == 1:
                result_helmet = Helmet_detection(helmet_portion,clf_helmet)
                if result_helmet == 1:
                    color = (0,255,0)
                    text = 'bike with helmet'
                else:
                    color = (255,0,0)
                    text = 'bike without helmet'
  #             The object is a bike....
  #             check for helmet....
            else:
                color = (255,255,255)
                text = 'non-bike'
                #The object is not a bike....
            img = cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
            cv2.putText(img,text,(x,y), cv2.FONT_HERSHEY_SIMPLEX , 1,color,1,cv2.LINE_AA)
            
    #cv2.imshow('BS',fgmask)
    cv2.imshow('image',img)
    k = cv2.waitKey(100)
    if k == ord('s'):
        break
cv2.destroyAllWindows()